**Bảng DIM**

In [7]:
import pandas as pd

table = {
    "1. Doanh thu bán hàng và cung cấp dịch vụ": "01",
    "2. Các khoản giảm trừ doanh thu": "02",
    "3. Doanh thu thuần về bán hàng và cung cấp dịch vụ (10 = 01 - 02)": "10",
    "4. Giá vốn hàng bán": "11",
    "5. Lợi nhuận gộp về bán hàng và cung cấp dịch vụ(20=10-11)": "20",
    "6. Doanh thu hoạt động tài chính": "21",
    "7. Chi phí tài chính": "22",
    "- Trong đó: Chi phí lãi vay": "23",
    "8. Phần lãi lỗ trong công ty liên doanh, liên kết": "24",
    "9. Chi phí bán hàng": "25",
    "10. Chi phí quản lý doanh nghiệp": "26",
    "11. Lợi nhuận thuần từ hoạt động kinh doanh{30=20+(21-22) + 24 - (25+26)}": "30",
    "12. Thu nhập khác": "31",
    "13. Chi phí khác": "32",
    "14. Lợi nhuận khác(40=31-32)": "40",
    "15. Tổng lợi nhuận kế toán trước thuế(50=30+40)": "50",
    "16. Chi phí thuế TNDN hiện hành": "51",
    "17. Chi phí thuế TNDN hoãn lại": "52",
    "18. Lợi nhuận sau thuế thu nhập doanh nghiệp(60=50-51-52)": "60",
    "19. Lợi nhuận sau thuế công ty mẹ": "428",
    "20. Lợi nhuận sau thuế công ty mẹ không kiểm soát": "429",
    "21. Lãi cơ bản trên cổ phiếu(*)": "70",
    "22. Lãi suy giảm trên cổ phiếu (*)": "71",
}


In [8]:
data = []

def flatten_dictionary(dictionary, prefix=''):
    for key, value in dictionary.items():
        if isinstance(value, dict):
            flatten_dictionary(value, prefix + key + ' - ')
        else:
            # Remove leading numbers and extra spaces between words
            stripped_key = ' '.join(filter(None, map(str.strip, key.split())))
            data.append((prefix + stripped_key, value))

flatten_dictionary(table)

df_dim = pd.DataFrame(data, columns=['Chỉ tiêu', 'Mã số'])
df_dim

,Chỉ tiêu,Mã số
0,1. Doanh thu bán hàng và cung cấp dịch vụ,01
1,2. Các khoản giảm trừ doanh thu,02
2,3. Doanh thu thuần về bán hàng và cung cấp dịc...,10
3,4. Giá vốn hàng bán,11
4,5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...,20
5,6. Doanh thu hoạt động tài chính,21
6,7. Chi phí tài chính,22
7,- Trong đó: Chi phí lãi vay,23
8,"8. Phần lãi lỗ trong công ty liên doanh, liên kết",24
9,9. Chi phí bán hàng,25


**BẢNG FACT**

In [9]:
# Đọc file Cân đối kế toán Quý 1
df1 = pd.read_excel('[aav] Quy1_KQHĐKD.xlsx')
df2=df

# Xoá đi các dòng chứa toàn giá trị NaN
df1=df1.dropna(how='all')

# Đặt tên cho cột đầu tiên là chỉ tiêu
df1.columns = ['Chỉ tiêu'] + list(df1.columns[1:])


# Gộp 2 dataframe theo chỉ tiêu
df_merged = pd.merge(df1, df2, on='Chỉ tiêu', how='outer')

# #Xoá cột "chỉ tiêu"
df_merged=df_merged.drop('Chỉ tiêu',axis=1)

# Sử dụng hàm melt()
df_merged = df_merged.melt(id_vars='Mã số', var_name='Quarter', value_name='value')

# Tách 'Quarter' thành 'year' và 'Quarter'
df_merged[['Quarter', 'year']] = df_merged['Quarter'].str.split(' - ', expand=True)

# Loại bỏ các ký tự không phải số trong cột 'Quarter'
df_merged['Quarter'] = df_merged['Quarter'].str.extract(r'(\d+)')

df_merged


,Mã số,Quarter,value,year
0,01,1,"88,774,255,428",2019
1,02,1,NaN,2019
2,10,1,"88,774,255,428",2019
3,11,1,"77,492,399,180",2019
4,20,1,"11,281,856,248",2019
...,...,...,...,...
110,60,1,"-3,348,195,576",2023
111,428,1,"-3,344,350,695",2023
112,429,1,"-3,844,881",2023
113,70,1,NaN,2023


In [20]:
ten_file_excel = 'dim&fact.xlsx'

# Tên của sheet bạn muốn tạo trong file Excel
ten_sheet = 'dim_KQHDKD'

# Xuất DataFrame vào file Excel
df_dim.to_excel(ten_file_excel, sheet_name=ten_sheet, index=False)

In [21]:
# Tên của file Excel bạn muốn sử dụng
ten_file_excel = 'dim&fact.xlsx'

# Tên của sheet bạn muốn thêm DataFrame mới
ten_sheet_moi = 'fact_KQHDKD'

# Ghi DataFrame thứ hai vào file Excel
with pd.ExcelWriter(ten_file_excel, engine='openpyxl', mode='a') as writer:
    df_merged.to_excel(writer, sheet_name=ten_sheet_moi, index=False)